In [1]:
# using Distributions
# using Plots
using ProfileView

Gtk-Message: 15:57:29.111: Failed to load module "canberra-gtk-module"


In [1]:
ms = 1e-3
mV = 1e-3
nF = 1e-9

# z = Normal(0, 1e-3)
fps = (3, 7)
dt = ms

struct NeuronParameters
    N::Int64
    Vₜ
    Vᵣ
    Cₘ
    Eᵢ
    Eₑ
    Eₘ
    Eₗ
    τᵣ
    τₛ
    τₘ
    kₛ
end

struct SynapseParameters
    n_exc
    n_inh
    w_exc
    w_inh
    w_exc_fp
    w_inh_fp
    fp_width
end

np = NeuronParameters(20, -48. *mV, -80. *mV, 1. *nF, -70. *mV, 0., -70. *mV, -70. *mV, 2.  *ms, 5. *ms, 5. *ms, 1/(5*ms*exp(-1.)))
sp = SynapseParameters(5, 7, 0.05, -0.10, 0.06, -0.25, 3)

SynapseParameters(5, 7, 0.05, -0.1, 0.06, -0.25, 3)

In [2]:
function make_connectivity_matrix(sp::SynapseParameters, np::NeuronParameters)
    cm = zeros(Float64, np.N,np.N)
    for i in 1:np.N
        for k in 1:np.N
            dist = abs(i - k)
            cm[i, k] = min(np.N - dist, dist)
        end
    end
    cm[1 .<= cm .<= sp.n_exc] .= sp.w_exc
    cm[cm .> sp.n_exc] .= sp.w_inh
    
    if length(fps) > 0
        for i in fps
            x = cm[i, :]
            x[x .== sp.w_exc] .= sp.w_exc_fp
            x[x .== sp.w_inh] .= sp.w_inh_fp
            cm[i, :] = x
        end
    end
    cm
end

make_connectivity_matrix (generic function with 1 method)

In [4]:
function dv(v, sd, cm)
    Iᵢ = zeros(Float64, np.N)
    Iₑ = zeros(Float64, np.N)

    for i in 1:np.N
        inh = Float32[]
        exc = Float32[]

        I_array = cm[:, i] .* sd .* np.kₛ .* exp.(-sd ./ np.τₛ) .* 1e-6
        for I in I_array
            if I < 0
                push!(inh, -I.*(v[i] - np.Eᵢ))
            else
                push!(exc, I * (v[i] - np.Eₑ))
            end
        end
        Iᵢ[i] = sum(inh)
        Iₑ[i] = sum(exc)
    end
    @show sum(Iᵢ .+ Iₑ)
    δv = (-np.Cₘ / np.τₘ .* (v .- np.Eₗ) .- Iᵢ .- Iₑ) ./ np.Cₘ .* dt # .+ rand(z, length(v))
    
    return δv+v
end

dv (generic function with 1 method)

In [5]:
function step(potentials, spike_delays, cm)
    t = size(potentials, 2)
    
    to_depolarize = potentials[:, t] .> np.Vₜ
    to_hyperpolarize = potentials[:, t] .== 0
    
    potentials = [potentials dv(potentials[:, t], spike_delays, cm)] # This looks inefficient
    tmp = potentials[:, t+1]
    tmp[to_depolarize] .= 0.0
    tmp[to_hyperpolarize] .= np.Vᵣ
    potentials[:, t+1] = tmp
    
    
    
    spike_delays .+= dt
    spike_delays[to_depolarize] .= 0.0

    potentials, spike_delays
end

step (generic function with 1 method)

In [11]:
function simulate(time)
    cm = make_connectivity_matrix(sp, np)
    pot = zeros(Float64, np.N) .+ np.Vᵣ
    spiked = zeros(Float64, np.N) .+ 2
    spiked[1:3] .= 0.1
    for t in 1:time
        pot, spiked = step(pot, spiked, cm)
    end
    
    pot
end

simulate (generic function with 1 method)

In [12]:
simulate(5)

sum(Iᵢ .+ Iₑ) = -1.888142465455519e-14
sum(Iᵢ .+ Iₑ) = -1.4566320025833706e-14
sum(Iᵢ .+ Iₑ) = -1.1351366912135672e-14
sum(Iᵢ .+ Iₑ) = -8.926734829304997e-15
sum(Iᵢ .+ Iₑ) = -7.076592686825989e-15


20×6 Array{Float64,2}:
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  -0.078  -0.0764  -0.07512  -0.074096  -0.0732768
 -0.08  

In [8]:
@profview simulate(2000)

Gtk.GtkWindowLeaf(name="", parent, width-request=-1, height-request=-1, visible=TRUE, sensitive=TRUE, app-paintable=FALSE, can-focus=FALSE, has-focus=FALSE, is-focus=FALSE, focus-on-click=TRUE, can-default=FALSE, has-default=FALSE, receives-default=FALSE, composite-child=FALSE, style, events=0, no-show-all=FALSE, has-tooltip=FALSE, tooltip-markup=NULL, tooltip-text=NULL, window, opacity=1.000000, double-buffered, halign=GTK_ALIGN_FILL, valign=GTK_ALIGN_FILL, margin-left, margin-right, margin-start=0, margin-end=0, margin-top=0, margin-bottom=0, margin=0, hexpand=FALSE, vexpand=FALSE, hexpand-set=FALSE, vexpand-set=FALSE, expand=FALSE, scale-factor=1, border-width=0, resize-mode, child, type=GTK_WINDOW_TOPLEVEL, title="Profile", role=NULL, resizable=TRUE, modal=FALSE, window-position=GTK_WIN_POS_NONE, default-width=800, default-height=600, destroy-with-parent=FALSE, hide-titlebar-when-maximized=FALSE, icon, icon-name=NULL, screen, type-hint=GDK_WINDOW_TYPE_HINT_NORMAL, skip-taskbar-hint

./In[4], dv: line 14
./In[4], dv: line 12
./In[4], dv: line 9


In [11]:
@time simulate(2000)

  0.653051 seconds (7.17 M allocations: 574.235 MiB, 7.51% gc time)


20×2001 Array{Float64,2}:
 -0.08  -0.078  -0.0753317  -0.0730996  …  -0.0623655  -0.0633253  -0.0620947
 -0.08  -0.078  -0.0753317  -0.0730996     -0.0589699  -0.0589401  -0.0566837
 -0.08  -0.078  -0.0753317  -0.0730996     -0.0563854  -0.0556558  -0.0525423
 -0.08  -0.078  -0.0739519  -0.0698905     -0.08       -0.0678515  -0.0558321
 -0.08  -0.078  -0.0725721  -0.0665808     -0.078      -0.0670171  -0.0537726
 -0.08  -0.078  -0.0711922  -0.0631705  …  -0.078      -0.0656373  -0.050413
 -0.08  -0.078  -0.0711922  -0.0631705     -0.078      -0.0659845  -0.0494228
 -0.08  -0.078  -0.0711922  -0.0631705     -0.078      -0.0656373  -0.048758
 -0.08  -0.078  -0.0729282  -0.0670272     -0.078      -0.0656373  -0.048758
 -0.08  -0.078  -0.0729282  -0.0670272     -0.078      -0.0656373  -0.050413
 -0.08  -0.078  -0.0729282  -0.0670272  …  -0.078      -0.0670171  -0.0537726
 -0.08  -0.078  -0.0711922  -0.0631705     -0.08       -0.0678515  -0.0558321
 -0.08  -0.078  -0.0711922  -0.0631705    

In [14]:
pots = simulate(100, true)
heatmap(pots)

LoadError: MethodError: no method matching simulate(::Int64, ::Bool)
Closest candidates are:
  simulate(::Any) at In[11]:1

In [13]:
spikes = pots .== 0
spikes = broadcast(*, spikes, [1:1:20;])
slice = spikes[:, 1:30]

slice = reshape(slice, :)
slice = slice[slice .> 0]

fit(Normal, slice)

LoadError: UndefVarError: pots not defined